In [1]:
import os
import skimage
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
import glob
from skimage import io
import seaborn as sns 
from skimage import color


from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_curve  
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
from itertools import combinations
import time

# IMAGE MATCHING FUNCTION

In [2]:
from sklearn.preprocessing import MinMaxScaler
PRED_SCORES=[]
TH=253
def ImageMatch(imgA , imgB , method='cc' , normalize='no'):
    
        if normalize=='yes':
            normA = MinMaxScaler()
            normB = MinMaxScaler()
            img_A = normA.fit_transform(imgA)
            img_B = normA.fit_transform(imgB)

# --------------------------------cc-----------------------------------#  
        if method == 'cc':
            cross_corr=0        
            for x in range(len(imgA)):    # x-> row of pixels of images
                for y in range(len(imgB[x])):   # y-> col of pixels of images
                    cross_corr += (imgA[x][y]*imgB[x][y])  # cross_Cor is the values of scores
            if cross_corr >= TH :  # similar
                PRED_SCORES.append(1)
            else:                  # not similar
                PRED_SCORES.append(0) 
            return cross_corr    
# --------------------------------conv-----------------------------------#                  
        elif method=='conv':
            conv=0
            fliped_imgA = np.flip(imgA, axis=0)  # flip the rows
            fliped_imgA_final = np.flip(fliped_imgA, axis=1) # flip the col
            for x in range(len(fliped_imgA_final)):    # x-> row of pixels of images
                for y in range(len(fliped_imgA_final[x])):   # y-> col of pixels of images
                    conv += (fliped_imgA_final[x][y]*imgB[x][y])  # cross_Cor is the values of scores
            
            if conv >= TH :  # similar
                PRED_SCORES.append(1)
            else:            # not similar
                PRED_SCORES.append(0) 
            return conv    
# --------------------------------ssd-----------------------------------#                
        elif method == 'ssd':
            ssd=0
            for x in range(len(imgA)):
                for y in range(len(imgB[x])):
                    ssd += np.square(imgA[x][y] - imgB[x][y])
            if ssd >= TH :  # not similar
                PRED_SCORES.append(0)
            else:            # similar
                PRED_SCORES.append(1) 
            return ssd    


# PREPROCESSING IMAGES , COMBINATION & LABELS OF TRAINING IMAGES

In [3]:
training_paths=[]   # paths of the training images 

training_pairs=[]   # similarity and nonsimilarity pairs of images 

training_labels=[]  # labels of similarity and nonsimilarity pairs of images


for (root,dirs,files) in os.walk('C:\\Users\\SOFT\\Desktop\\Deep learning section\\training2' ,topdown=True):
    for i in files :
        path=os.path.join(root,i)
        training_paths.append(path)
        
training_pairs = list(combinations(training_paths, 2))  

for i in training_pairs:
    img1 =os.path.dirname(i[0])
    img2=os.path.dirname(i[1])
    if(img1==img2):
        training_labels.append(1)
    else :
        training_labels.append(0)
# print(Training_Labels)        
# print(len(training_labels))


#-------------------------------------------------------------------------#

Training_Images_path=[]
training_pairs.clear()
for i in training_paths:
    img = io.imread( i )
    img_resized = resize( img , (32 , 32 , 3 ) )  # 32*32*3
    img_gray = color.rgb2gray( img_resized  )     # 32*32
    Training_Images_path.append(img_gray)
training_pairs = list(combinations(Training_Images_path, 2)) 
 


    

# SCORES OF TRAINING IMAGES

In [4]:
training_scores=[]  # scores of the training images 
for i in training_pairs:
    img1=i[0]
    img2=i[1]
    result_train_score=ImageMatch(img1 , img2 ,'cc','no')
    #print(result_train_score)
    training_scores.append(result_train_score)
    

In [5]:
predict_train_label=[]
for j in training_scores:
    if j<=TH:
        predict_train_label.append(1)
    else:
        predict_train_label.append(0)
        

# ACCURACY  , PRECISION , RECAAL , F1 SCORE OF TRAINING

In [6]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics


conf = confusion_matrix(training_labels , predict_train_label)
print('confusion matrix',"\n",conf)

# Model Accuracy: how often is the classifier correct?

print("Accuracy:",metrics.accuracy_score(training_labels , predict_train_label))

print("precition:",metrics.precision_score(training_labels , predict_train_label))

print("f1 score:",metrics.f1_score(training_labels , predict_train_label))

print("recall:",metrics.recall_score(training_labels , predict_train_label))


confusion matrix 
 [[13493     7]
 [ 2606     4]]
Accuracy: 0.8378026070763501
precition: 0.36363636363636365
f1 score: 0.003052270125906143
recall: 0.0015325670498084292


# PREPROCESSING IMAGES , COMBINATION & LABELS OF TESTING IMAGES

In [7]:
testing_paths=[]   # paths of the training images 

testing_pairs=[]   # similarity and nonsimilarity pairs of images 

testing_labels=[]  # labels of similarity and nonsimilarity pairs of images


for (root,dirs,files) in os.walk('C:\\Users\\SOFT\\Desktop\\Deep learning section\\testing2' ,topdown=True):
    for i in files :
        path=os.path.join(root,i)
        testing_paths.append(path)
        
testing_pairs = list(combinations(testing_paths, 2))  

for i in testing_pairs:
    img1 =os.path.dirname(i[0])
    img2=os.path.dirname(i[1])
    if(img1==img2):
        testing_labels.append(1)
    else :
        testing_labels.append(0)
# print(testing_Labels)        
#print(len(testing_labels))


#-------------------------------------------------------------------------#

Testing_Images_path=[]
testing_pairs.clear()
for i in testing_paths:
    img = io.imread( i )
    img_resized = resize( img , (32 , 32 , 3 ) )  # 32*32*3
    img_gray = color.rgb2gray( img_resized  )     # 32*32
    Testing_Images_path.append(img_gray)
testing_pairs = list(combinations(Testing_Images_path, 2)) 
 

# SCORES OF TESTING IMAGES

In [8]:
testing_scores=[]  # scores of the training images 
starttime = time.time()
for i in testing_pairs:
    img1=i[0]
    img2=i[1]
    result_test_score=ImageMatch(img1 , img2 ,'cc','no')
    #print(result_test_score)
    testing_scores.append(result_test_score)
endtime = time.time()
    

In [9]:
predict_test_label=[]
for j in testing_scores:
    if j<=TH:
        predict_test_label.append(1)
    else:
        predict_test_label.append(0)

In [10]:
EX_time = endtime - starttime 
print("EX_time of ImageMatching = ",EX_time)

EX_time of ImageMatching =  1.2147493362426758


In [11]:
# cc "image matching function " takes long time than MLP


# ACCURACY  , PRECISION , RECAAL , F1 SCORE OF TESTING

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics


conf = confusion_matrix(testing_labels , predict_test_label)
print('confusion matrix',"\n",conf)

# Model Accuracy: how often is the classifier correct?

print("Accuracy:",metrics.accuracy_score(testing_labels , predict_test_label))

print("precition:",metrics.precision_score(testing_labels , predict_test_label))

print("f1 score:",metrics.f1_score(testing_labels , predict_test_label))

print("recall:",metrics.recall_score(testing_labels , predict_test_label))


confusion matrix 
 [[1500    0]
 [ 269    1]]
Accuracy: 0.8480225988700565
precition: 1.0
f1 score: 0.007380073800738007
recall: 0.003703703703703704


In [13]:
# cc "image matching function " better accuracy than MLP

# PREPROCESSING FOR MLP

In [14]:
# flatten of train
x_train=[]
for x in training_pairs:
    arr_x=np.array(x)      # i should convert it to array cause tuple don't have function (reshape)
    Tra_Resh_x=arr_x.reshape(2048)
    x_train.append(Tra_Resh_x)

In [15]:
#flatten of test
x_test=[]
for x in testing_pairs:
    arr_x=np.array(x)      # i should convert it to array cause tuple don't have function (reshape)
    Tes_Resh_x=arr_x.reshape(2048)
    x_test.append(Tes_Resh_x)

In [16]:
# label of train 
y_train=[]
for y in training_labels:
    y_train.append(y)

In [17]:
# label of test 
y_test=[]
for y in testing_labels:
    y_test.append(y)

# MLP 

In [18]:
from sklearn.neural_network import MLPClassifier
MLPcls = MLPClassifier()
MLP = MLPcls.fit(x_train, y_train)

In [19]:
predict_y_train =  MLP.predict( x_train )
start_time = time.time()
predict_y_test =  MLP.predict( x_test  )
end_time = time.time()

In [20]:
print("Accuracy of training = " ,accuracy_score( y_train ,  predict_y_train ))
print("Accuracy of testing = ", accuracy_score( y_test ,  predict_y_test  ))

Accuracy of training =  0.9835505896958411
Accuracy of testing =  0.5203389830508475


In [21]:
EX_time = end_time - start_time 
print("EX_time of MLP = ",EX_time)

EX_time of MLP =  1.008303165435791


In [22]:
# MLP time is better than ImageMatch function 

In [23]:
conf = confusion_matrix(y_test , predict_y_test)
print('confusion matrix of testing ',"\n",conf)

# Model Accuracy: how often is the classifier correct?

print("Accuracy of testing:",metrics.accuracy_score(y_test , predict_y_test))

print("precition of testing:",metrics.precision_score(y_test , predict_y_test))

print("f1 score of testing:",metrics.f1_score(y_test , predict_y_test))

print("recall of testing:",metrics.recall_score(y_test , predict_y_test))

confusion matrix of testing  
 [[780 720]
 [129 141]]
Accuracy of testing: 0.5203389830508475
precition of testing: 0.16376306620209058
f1 score of testing: 0.2493368700265252
recall of testing: 0.5222222222222223


In [24]:
 # MLP low accuracy than ImageMatch 